## Kernelized Regression with Missing Features
In this task, we predict electricity prices in Switzerland using price information from other countries and additional features. Specifically, we are provided with electricity prices for certain seasons (spring, summer, autumn, winter) over the past few years in Switzerland and some other countries. Additionally, we need to address missing features in our dataset.

#### Imports & Reading Input

In [39]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, RBF, Matern, RationalQuadratic
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

data_train = train_df.to_numpy()
data_test = test_df.to_numpy() # note that data_test does not contain price_CHF column
print('Training Data: \n', data_train)

Training Data: 
 [['spring' nan 9.644027877268496 ... -3.931031226630509 nan
  -3.238196806151894]
 ['summer' nan 7.246060839801865 ... nan nan -3.212894038068976]
 ['autumn' -2.101936612395246 7.620084525124941 ... -4.07384968174626 nan
  -3.1140608060213903]
 ...
 ['summer' -0.9711569246205298 nan ... -1.4993613445447886
  3.110638067512592 2.230252561735496]
 ['autumn' nan nan ... -1.5477160129737388 3.105416529245648
  1.989139721317721]
 ['winter' -1.0583425835760634 nan ... nan 3.272815718725681
  2.080666809994271]]


#### Data Preprocessing
Typical imputation strategies to deal with missing data include: 
- discarding rows or columns with missing data
- replacing missing values with the corresponding mean or median
- advanced imputation strategies based on iterative model fitting

References (data imputation): 
- kaggle notebook: https://www.kaggle.com/code/residentmario/simple-techniques-for-missing-data-imputation/notebook
- sklearn website: https://scikit-learn.org/stable/modules/impute.html

References (handling categorical data):
- kaggle notebook: https://www.kaggle.com/code/alexisbcook/categorical-variables
- sklearn website: https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features



#### Categorical Encoding
We use ordinal encoding to deal with the categorical variables, i.e. we use the following mapping: spring $\mapsto 1.0$, summer $\mapsto 2.0$, autumn $\mapsto 3.0$, winter $\mapsto 4.0$. Scikit-learn has a `OrdinalEncoder` class that can be used to obtain the ordinal encodings. 

In [40]:
# categorical encoding
categorical_column_train = data_train[:, 0].reshape(-1, 1)  
numerical_columns_train = data_train[:, 1:]  
categorical_column_test = data_test[:, 0].reshape(-1, 1)
numerical_columns_test = data_test[:, 1:]

encoder = OrdinalEncoder()
encoded_categorical_train = encoder.fit_transform(categorical_column_train)
encoded_categorical_test = encoder.fit_transform(categorical_column_test)

# Concatenate the encoded categorical column with the numerical columns
data_train = np.hstack((encoded_categorical_train, numerical_columns_train))
data_test = np.hstack((encoded_categorical_test, numerical_columns_test))
print('Training Data: \n', data_train)

Training Data: 
 [[1.0 nan 9.644027877268496 ... -3.931031226630509 nan -3.238196806151894]
 [2.0 nan 7.246060839801865 ... nan nan -3.212894038068976]
 [0.0 -2.101936612395246 7.620084525124941 ... -4.07384968174626 nan
  -3.1140608060213903]
 ...
 [2.0 -0.9711569246205298 nan ... -1.4993613445447886 3.110638067512592
  2.230252561735496]
 [0.0 nan nan ... -1.5477160129737388 3.105416529245648 1.989139721317721]
 [3.0 -1.0583425835760634 nan ... nan 3.272815718725681 2.080666809994271]]


#### Data Imputation
We use `IterativeImputer` from Scikit-kearn to impute missing values. It fills in missing values iteratively by modeling each feature with missing values as a function of the other features. This is done using a regression model that predicts the missing values based on the available data. The imputation process involves mulitple iterations. In each iteration, the algorithm estimates the missing values, update the dataset with these estimates, and then refines the estimates in subsequent iterations. This is done until convergence or the maximum number of iterations is reached. 

By modeling each feature as a function of others, `IterativeImputer` can capture complex relationships between features and provide more accurate imputations compared to methods like mean imputation. 

In [41]:
# data imputation
imp = IterativeImputer(max_iter=100, random_state=0)
data_train = imp.fit_transform(data_train)
data_test = imp.fit_transform(data_test)
print('Training Data: \n', data_train)

Training Data: 
 [[ 1.         -1.94730966  9.64402788 ... -3.93103123 -2.6863533
  -3.23819681]
 [ 2.         -2.17086395  7.24606084 ... -3.99871774 -2.4877255
  -3.21289404]
 [ 0.         -2.10193661  7.62008453 ... -4.07384968 -2.43497284
  -3.11406081]
 ...
 [ 2.         -0.97115692 -0.44059049 ... -1.49936134  3.11063807
   2.23025256]
 [ 0.         -1.14346076 -0.33911152 ... -1.54771601  3.10541653
   1.98913972]
 [ 3.         -1.05834258 -0.88944359 ... -1.26471153  3.27281572
   2.08066681]]


#### Modeling and Prediction
We aim to use a kernelized estimator, with the challenge being to select the appropriate kernel for the regression. Commonly used kernels include linear, squared exponential (RBF), polynomial, Matern, and RationalQuadratic kernels. To determine the optimal kernel, we apply the $K$-fold cross-validation technique. After identifying the optimal kernel, we train our model on the entire training set and make final predictions on the test set.

We use the `GaussianProcessRegressor` framework for training and making predictions.

Note that we use the R²-score metric to assess the quality of our model, which is defined as:
$$R^2(y^*, y) = 1 - \frac{\sum_{i=1}^N (y_i - y_i^*)^2}{\sum_{i=1}^N(y_i - \bar y)^2},$$
where $\bar y$ denotes the average of the true values $y_i$.

In [42]:
X_train = np.delete(data_train, 2, axis=1)
y_train = data_train[:, 2]
X_test = data_test

K = 10  # number of folds
kernels = [DotProduct(), RBF(), Matern(), RationalQuadratic()]
Error_mat = np.zeros((K, len(kernels)))
kf = KFold(n_splits=K)

# Cross-validation loop
for i, (train, test) in enumerate(kf.split(X_train)):
    for j, kernel in enumerate(kernels): 
        # training model 
        X_train_cv, y_train_cv = X_train[train], y_train[train]
        gpr = GaussianProcessRegressor(kernel=kernel)
        gpr.fit(X_train_cv, y_train_cv)

        # testing model 
        X_test_cv, y_test_cv = X_train[test], y_train[test]
        y_pred_cv = gpr.predict(X_test_cv)
        Error_mat[i,j] = r2_score(y_test_cv, y_pred_cv)


# Averaging R2 scores across folds for each kernel
avg_r2_score = np.mean(Error_mat, axis=0)
print('Average R2-score:', avg_r2_score)

# figuring out the optimal kernel
max = np.argmax(avg_r2_score)
kernel_opt = kernels[max]
print('Optimal Kernel:', kernel_opt)

# training final model with optimal kernel on full training set
gpr = GaussianProcessRegressor(kernel=kernel_opt)
gpr.fit(X_train, y_train)
# making prediction on test set
y_pred = gpr.predict(X_test)

Average R2-score: [1.         0.81978014 0.85539344 0.85409033]
Optimal Kernel: DotProduct(sigma_0=1)


#### Saving Results

In [28]:
dt = pd.DataFrame(y_pred) 
dt.columns = ['price_CHF']
dt.to_csv('results.csv', index=False)
print("\nResults file successfully generated!")


Results file successfully generated!
